In [22]:
import yfinance as yf
import numpy as np
from datetime import datetime
from math import log, sqrt, exp
from scipy.stats import norm

In [23]:
# Jupyter Notebook cell 2: Black–Scholes Formulas

def black_scholes_call(S, K, T, r, sigma):
    """
    Black-Scholes price of a European Call.
    
    S: Current underlying price
    K: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate (annualized, decimal form)
    sigma: Volatility (annualized, decimal form)
    """
    if T <= 0 or sigma <= 0:
        # Extremely rough edge case
        return max(0.0, S - K)
    
    d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    
    call_price = S * norm.cdf(d1) - K * exp(-r*T) * norm.cdf(d2)
    return call_price


def black_scholes_put(S, K, T, r, sigma):
    """
    Black-Scholes price of a European Put.
    
    S: Current underlying price
    K: Strike price
    T: Time to maturity (in years)
    r: Risk-free interest rate (annualized, decimal form)
    sigma: Volatility (annualized, decimal form)
    """
    if T <= 0 or sigma <= 0:
        # Extremely rough edge case
        return max(0.0, K - S)
    
    d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    
    # Using put-call parity approach or direct formula:
    put_price = K * exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price


In [24]:
# Jupyter Notebook cell 3: Implied Vol Solvers (Newton–Raphson)

def implied_vol_call(market_price, S, K, T, r=0.0, tol=1e-6, max_iter=100):
    """
    Newton-Raphson to find implied volatility for a European Call.
    market_price: Observed market price (bid/ask mid or last)
    S, K, T, r: Underlying price, strike, time-to-expiry, risk-free rate
    """
    # Intrinsic value for a call is max(0, S - K*exp(-r*T))
    intrinsic_value = max(0.0, S - K*exp(-r*T))
    if market_price < intrinsic_value:
        return np.nan  # Impossible if below intrinsic
    
    sigma = 0.3  # initial guess
    for _ in range(max_iter):
        price_guess = black_scholes_call(S, K, T, r, sigma)
        diff = price_guess - market_price
        
        if abs(diff) < tol:
            return sigma
        
        # Vega: partial derivative wrt sigma
        d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * sqrt(T))
        vega = S * norm.pdf(d1) * sqrt(T)
        
        if vega < 1e-8:  # avoid division by near-zero
            break
        
        sigma -= diff / vega
        if sigma < 1e-8:
            sigma = 1e-8
    
    return sigma


def implied_vol_put(market_price, S, K, T, r=0.0, tol=1e-6, max_iter=100):
    """
    Newton-Raphson to find implied volatility for a European Put.
    market_price: Observed market price (bid/ask mid or last)
    S, K, T, r: Underlying price, strike, time-to-expiry, risk-free rate
    """
    # Intrinsic value for a put is max(0, K*exp(-r*T) - S)
    intrinsic_value = max(0.0, K*exp(-r*T) - S)
    if market_price < intrinsic_value:
        return np.nan
    
    sigma = 0.3  # initial guess
    for _ in range(max_iter):
        price_guess = black_scholes_put(S, K, T, r, sigma)
        diff = price_guess - market_price
        
        if abs(diff) < tol:
            return sigma
        
        # Vega for put is the same magnitude as call: partial derivative wrt sigma
        d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * sqrt(T))
        vega = S * norm.pdf(d1) * sqrt(T)
        
        if vega < 1e-8:
            break
        
        sigma -= diff / vega
        if sigma < 1e-8:
            sigma = 1e-8
    
    return sigma



In [25]:
# Jupyter Notebook cell 4: Get Risk-Free Rate from T-Bill

def get_risk_free_rate():
    """
    Fetches the 3-month T-bill (annualized) yield from yfinance
    using the '^IRX' ticker. You can switch to a different T-Bill
    if you prefer e.g. '^IRX' -> 3-month, '^FVX' -> 5-year, etc.
    
    Returns the yield in decimal form (e.g. 0.045 => 4.5%).
    """
    tbill = yf.Ticker("^IRX")  # 13-week T-bill index on Yahoo
    hist = tbill.history(period="1d")
    if hist.empty:
        # Fallback if no data
        return 0.042
    
    # ^IRX is often quoted in basis points or %; typically it's in hundredths
    last_close = hist["Close"].iloc[-1]
    # Convert to decimal form (e.g. 4.5 => 0.045)
    r_decimal = last_close / 100.0
    return r_decimal


In [26]:
# Jupyter Notebook cell 5: Get Option Data

def get_option_data(ticker_str, contract_type="calls"):
    """
    Retrieves option chain data (calls or puts) for a given ticker's first expiration.
    
    contract_type: 'calls' or 'puts'
    Returns:
      - DataFrame of calls/puts
      - Underlying spot price S
      - Time to expiration (in years)
    """
    ticker = yf.Ticker(ticker_str)
    
    # Get list of available expirations
    expirations = ticker.options
    if not expirations:
        raise ValueError(f"No option data available for ticker {ticker_str}.")
    
    # Use the earliest expiration
    first_expiry = expirations[0]
    
    # Retrieve calls or puts
    chain = ticker.option_chain(first_expiry)
    if contract_type == "calls":
        options_df = chain.calls
    elif contract_type == "puts":
        options_df = chain.puts
    else:
        raise ValueError("contract_type must be either 'calls' or 'puts'.")
    
    # Current underlying price
    stock_data = ticker.history(period="1d")
    if stock_data.empty:
        raise ValueError("Could not retrieve stock price history.")
    S = stock_data["Close"].iloc[-1]
    
    # Compute time to expiration in years
    expiry_dt = datetime.strptime(first_expiry, "%Y-%m-%d")
    now = datetime.now()
    T = (expiry_dt - now).days / 365.0
    if T <= 0:
        raise ValueError("Nearest option expiration is today or in the past.")
    
    return options_df, S, T


In [27]:
# Cell 6: Compute Implied Vols for All Contracts, Filter out NaN

def compute_implied_vols(ticker_str, contract_type="calls"):
    """
    Main function to:
      1. Get the risk-free rate from T-Bill yield.
      2. Retrieve the chosen option type (calls or puts) from yfinance.
      3. Compute implied volatility for each strike.
      4. Switch the moneyness formula:
         - For calls: moneyness = S/K
         - For puts:  moneyness = K/S
      5. Filter out contracts that return NaN for IV.
      6. Return arrays for implied vol, moneyness, time-to-expiry.
    """
    # 1) Risk-free rate
    r = get_risk_free_rate()
    
    # 2) Get option data
    options_df, S, T = get_option_data(ticker_str, contract_type=contract_type)
    
    # Prepare empty lists
    ivs = []
    mny = []
    ttes = []
    
    # 3) Loop over each row in the DataFrame
    for _, row in options_df.iterrows():
        K = row["strike"]
        
        # Mid-price from bid/ask if possible
        bid = row.get("bid", np.nan)
        ask = row.get("ask", np.nan)
        if not np.isnan(bid) and not np.isnan(ask):
            market_price = 0.5 * (bid + ask)
        else:
            market_price = row.get("lastPrice", np.nan)
        
        if np.isnan(market_price):
            continue  # skip if we have no usable price
        
        # 4) Compute implied volatility & moneyness
        if contract_type == "calls":
            iv = implied_vol_call(market_price, S, K, T, r=r)
            money = S / K  # calls: S/K
        else:
            iv = implied_vol_put(market_price, S, K, T, r=r)
            money = K / S  # puts: K/S
        
        # 5) Filter out NaN
        if np.isnan(iv):
            continue
        
        ivs.append(iv)
        mny.append(money)
        ttes.append(T)
    
    return ivs, mny, ttes


In [28]:
# Jupyter Notebook cell 7: Example Usage

# Choose a ticker symbol
ticker_symbol = "AAPL"

# Compute implied vols, moneyness, time-to-expiry for CALLS
ivs_calls, mny_calls, ttes_calls = compute_implied_vols(ticker_symbol, contract_type="calls")

print("CALLS:")
print("Implied Volatilities:", ivs_calls)
print("Moneyness (S/K):", mny_calls)
print("Time to Expiration (years):", ttes_calls)

print("-"*60)

# Compute implied vols, moneyness, time-to-expiry for PUTS
ivs_puts, mny_puts, ttes_puts = compute_implied_vols(ticker_symbol, contract_type="puts")

print("PUTS:")
print("Implied Volatilities:", ivs_puts)
print("Moneyness (K/S):", mny_puts)
print("Time to Expiration (years):", ttes_puts)


CALLS:
Implied Volatilities: [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 827040.2136325614, 101725.6183571046, 2916.485855141354, 293.27040522959834, 1e-08, 0.6377603446445339, 0.565871149388399, 0.5073900556931948, 0.4595423476333842, 0.42068741820683164, 0.39682991949453544, 0.37765072861777566, 0.358631520645213, 0.3460988978461822, 0.3431213050998204, 0.34301043106588325, 0.3444035799432438, 0.3673495000400956, 0.40216229545790516, 0.4254348074684854, 0.47196947737009926, 0.46343683263604196, 1e-08, 878.5245739219343, 9210.86920352666, 113081.93059443333, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
Moneyness (S/K): [5.831999969482422, 5.183999972873264, 3.887999979654948, 2.332799987792969, 2.1207272616299715, 2.028521728515625, 1.943999989827474, 1.866239990234375, 1.7944615290715145, 1.7279999909577546, 1.6088275777882544, 1.5551